In [ ]:
!pip install transformer_lens

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.7/739.7 kB 61.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 105.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 89.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 64.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00


In [ ]:

import torch
from transformer_lens import HookedTransformer
from transformer_lens.utils import test_prompt

In [ ]:
import os, json
from typing import Union, Optional, Dict, Any, List, Tuple
from rich import print as rprint

In [ ]:
import os
from huggingface_hub import login

login(os.getenv("HF_TOKEN"))


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
drive_dir = "/content/drive/MyDrive/master_thesis/models"

import json, torch
from transformer_lens import HookedTransformer, HookedTransformerConfig

with open(f"{drive_dir}/config.json") as f:
    cfg_dict = json.load(f)

cfg = HookedTransformerConfig.from_dict(cfg_dict)
cfg.dtype = torch.float16

model = HookedTransformer(cfg)

state = torch.load(f"{drive_dir}/pytorch_model.bin", map_location="cpu")
model.load_state_dict(state)


if torch.cuda.is_available():
    torch.cuda.empty_cache()
    model = model.to(torch.float16).to("cuda")

print("Modell in float16 auf GPU – konsistente Dtypes.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Changing model dtype to torch.float16
Moving model to device:  cuda
Modell in float16 auf GPU – konsistente Dtypes.


In [ ]:
import os
import json
import torch
from typing import Dict, Any, Union, Optional, List


def new_test_prompt(
    prompt: str,
    answer: Union[str, List[str]],
    model,
    relation_name: str,
    json_output_folder: str,
    prepend_space_to_answer: bool = True,
    print_details: bool = True,
    prepend_bos: Optional[bool] = False,
    top_k: int = 10,
) -> Dict[str, Any]:
    """
    Lässt das Modell autoregressiv weiter‑dekodieren und evaluiert dabei
    Top‑1‑ und Top‑10‑Accuracy auf Token‑Ebene.

    Rückgabe:
        {
            "top_1_accuracy":  ...,
            "top_10_accuracy": ...,
            "first_token_prob": ...,
            "second_token_prob": ...,
            "is_correct":       True/False
        }
    """


    # Ordner & Pfad für JSON‑Logging vorbereiten

    os.makedirs(json_output_folder, exist_ok=True)
    json_path = os.path.join(
        json_output_folder, relation_name.replace(" ", "_").lower() + ".json"
    )


    # Referenz‑Antwort vorbereiten

    answers: List[str] = [answer] if isinstance(answer, str) else list(answer)
    if prepend_space_to_answer:
        answers = [a if a.startswith(" ") else " " + a for a in answers]

    gold_answer = answers[0]
    gold_answer_ids = model.to_tokens(gold_answer, prepend_bos=False)[0]
    gold_answer_str_tokens = model.to_str_tokens(gold_answer, prepend_bos=False)


    # Prompt tokenisieren

    context_ids = model.to_tokens(prompt, prepend_bos=prepend_bos)[0].clone()
    _ = model.to_str_tokens(prompt, prepend_bos=prepend_bos)


    # Autoregressive Schleife

    first_token_prob = second_token_prob = None
    top_1_correct = top_10_correct = 0
    first_token_correct: Optional[bool] = None   # Flag, ob 1. Token korrekt war
    entries = []

    for pos in range(len(gold_answer_ids)):

        # Forward‑Pass
        logits = model(context_ids.unsqueeze(0))
        probs = logits.softmax(dim=-1)

        next_token_logits = logits[0, -1]
        next_token_probs = probs[0, -1]

        sorted_probs, sorted_indices = next_token_probs.sort(descending=True)

        # Wahrscheinlichkeiten der Top‑2‑Tokens merken
        if first_token_prob is None:
            first_token_prob = sorted_probs[0].item()
            second_token_prob = sorted_probs[1].item()

        # Predicted Token + Vergleich mit Gold
        predicted_token_id = sorted_indices[0].item()
        predicted_token_str = model.to_string(predicted_token_id)

        gold_token_id = gold_answer_ids[pos].item()
        gold_token_str = gold_answer_str_tokens[pos]

        #print("Indizies top k",sorted_indices[:top_k])

        # Accuracy‑Zählung
        if predicted_token_id == gold_token_id:
            top_1_correct += 1
            #print(predicted_token_str, gold_token_str)
        if (sorted_indices[:top_k] == gold_token_id).any():
            top_10_correct += 1


        # Flag für korrektes 1. Token setzen
        if pos == 0:
            first_token_correct = (predicted_token_id == gold_token_id)

        # Logging‑Eintrag
        entries.append(
            {
                "prompt": prompt,
                "generated_prefix": model.to_string(context_ids.tolist()),
                "gold_token": gold_token_str,
                "predicted_top1_token": predicted_token_str,
                "token_pos_in_answer": pos,
                "gold_logit": next_token_logits[gold_token_id].item(),
                "gold_prob_percent": round(next_token_probs[gold_token_id].item() * 100, 2),
                "first_token_prob": first_token_prob,
                "second_token_prob": second_token_prob,
                "is_top_1": int(predicted_token_id == gold_token_id),
                "top_k_predictions": [
                    {
                        "rank": k,
                        "token": model.to_string(sorted_indices[k].item()),
                        "logit": next_token_logits[sorted_indices[k]].item(),
                        "probability_percent": round(sorted_probs[k].item() * 100, 2),
                    }
                    for k in range(min(top_k, sorted_indices.shape[0]))
                ],
            }
        )

        # Kontext erweitern (Greedy)
        context_ids = torch.cat(
            [
                context_ids,
                torch.tensor(
                    [predicted_token_id],
                    dtype=context_ids.dtype,
                    device=context_ids.device,
                ),
            ]
        )

        if print_details:
            print(
                f"[Step {pos}] Gold: |{gold_token_str}| "
                f"Pred: |{predicted_token_str}|  "
                f"Top‑1 Prob: {sorted_probs[0].item():.2%}"
            )


    # Accuracy berechnen & Ergebnisse speichern

    total_tokens = len(gold_answer_ids)

    # Falls das 1. Sub‑Token falsch ist, setzen wir nur Top‑1‑Accuracy auf 0
    if first_token_correct is False:
        top_1_accuracy = 0.0
        is_correct_word = False
    else:
        top_1_accuracy = 100 * top_1_correct / total_tokens
        is_correct_word = top_1_correct == total_tokens

    # Top‑10‑Accuracy wird immer unabhängig vom ersten Token berechnet
    top_10_accuracy = 100 * top_10_correct / total_tokens


    try:
        with open(json_path, "r", encoding="utf-8") as fh:
            history = json.load(fh)
            if not isinstance(history, list):
                history = []
    except FileNotFoundError:
        history = []

    history.extend(entries)
    with open(json_path, "w", encoding="utf-8") as fh:
        json.dump(history, fh, indent=2, ensure_ascii=False)

    return {
        "top_1_accuracy": top_1_accuracy,
        "top_10_accuracy": top_10_accuracy,
        "first_token_prob": first_token_prob,
        "second_token_prob": second_token_prob,
        "is_correct": is_correct_word,
    }


In [ ]:
import os
import json
import torch
from typing import Dict, Any, Union, Optional, List


def new_test_prompt_(

    prompt: str,
    answer: Union[str, List[str]],
    model,
    relation_name: str,
    #json_output_folder: str,
    prepend_space_to_answer: bool = True,
    print_details: bool = True,
    prepend_bos: Optional[bool] = False,
    top_k: int = 10,
) -> Dict[str, Any]:
    """
    Hat genau dieselbe Funktion wie oben, nur werden die Logits nicht in den JSON-Daten gespeichert.
    Wird ausschließlich zur Evaluation der Templates verwendet.


    """


    # Ordner & Pfad für JSON‑Logging vorbereiten

    #os.makedirs(json_output_folder, exist_ok=True)
    #json_path = os.path.join(
    #    json_output_folder, relation_name.replace(" ", "_").lower() + ".json"
    #)


    # Referenz‑Antwort vorbereiten

    answers: List[str] = [answer] if isinstance(answer, str) else list(answer)
    if prepend_space_to_answer:
        answers = [a if a.startswith(" ") else " " + a for a in answers]

    gold_answer = answers[0]
    gold_answer_ids = model.to_tokens(gold_answer, prepend_bos=False)[0]
    gold_answer_str_tokens = model.to_str_tokens(gold_answer, prepend_bos=False)


    # Prompt tokenisieren

    context_ids = model.to_tokens(prompt, prepend_bos=prepend_bos)[0].clone()
    _ = model.to_str_tokens(prompt, prepend_bos=prepend_bos)


    # Autoregressive Schleife

    first_token_prob = second_token_prob = None
    top_1_correct = top_10_correct = 0
    first_token_correct: Optional[bool] = None   # Flag, ob 1. Token korrekt war
    #entries = []

    for pos in range(len(gold_answer_ids)):

        # Forward‑Pass
        logits = model(context_ids.unsqueeze(0))
        probs = logits.softmax(dim=-1)

        next_token_logits = logits[0, -1]
        next_token_probs = probs[0, -1]

        sorted_probs, sorted_indices = next_token_probs.sort(descending=True)

        # Wahrscheinlichkeiten der Top‑2‑Tokens merken
        if first_token_prob is None:
            first_token_prob = sorted_probs[0].item()
            second_token_prob = sorted_probs[1].item()

        # Predicted Token + Vergleich mit Gold
        predicted_token_id = sorted_indices[0].item()
        predicted_token_str = model.to_string(predicted_token_id)

        gold_token_id = gold_answer_ids[pos].item()
        gold_token_str = gold_answer_str_tokens[pos]



        # Accuracy‑Zählung
        if predicted_token_id == gold_token_id:
            top_1_correct += 1
            #print(predicted_token_str, gold_token_str)
        if (sorted_indices[:top_k] == gold_token_id).any():
            top_10_correct += 1


        # Flag für korrektes 1. Token setzen
        if pos == 0:
            first_token_correct = (predicted_token_id == gold_token_id)


        """entries.append(
            {
                "prompt": prompt,
                "generated_prefix": model.to_string(context_ids.tolist()),
                "gold_token": gold_token_str,
                "predicted_top1_token": predicted_token_str,
                "token_pos_in_answer": pos,
                "gold_logit": next_token_logits[gold_token_id].item(),
                "gold_prob_percent": round(next_token_probs[gold_token_id].item() * 100, 2),
                "first_token_prob": first_token_prob,
                "second_token_prob": second_token_prob,
                "is_top_1": int(predicted_token_id == gold_token_id),
                "top_k_predictions": [
                    {
                        "rank": k,
                        "token": model.to_string(sorted_indices[k].item()),
                        "logit": next_token_logits[sorted_indices[k]].item(),
                        "probability_percent": round(sorted_probs[k].item() * 100, 2),
                    }
                    for k in range(min(top_k, sorted_indices.shape[0]))
                ],
            }
        )
        """
        # Kontext erweitern (Greedy)
        context_ids = torch.cat(
            [
                context_ids,
                torch.tensor(
                    [predicted_token_id],
                    dtype=context_ids.dtype,
                    device=context_ids.device,
                ),
            ]
        )

        if print_details:
            print(
                f"[Step {pos}] Gold: |{gold_token_str}| "
                f"Pred: |{predicted_token_str}|  "
                f"Top‑1 Prob: {sorted_probs[0].item():.2%}"
            )


    # Accuracy berechnen & Ergebnisse speichern

    total_tokens = len(gold_answer_ids)

    # Falls das 1. Sub‑Token falsch ist, setzen wir nur Top‑1‑Accuracy auf 0
    if first_token_correct is False:
        top_1_accuracy = 0.0
        is_correct_word = False
    else:
        top_1_accuracy = 100 * top_1_correct / total_tokens
        is_correct_word = top_1_correct == total_tokens

    # Top‑10‑Accuracy wird immer unabhängig vom ersten Token berechnet
    top_10_accuracy = 100 * top_10_correct / total_tokens

    # JSON‑Historie anhängen
    """try:
        with open(json_path, "r", encoding="utf-8") as fh:
            history = json.load(fh)
            if not isinstance(history, list):
                history = []
    except FileNotFoundError:
        history = []

    history.extend(entries)
    with open(json_path, "w", encoding="utf-8") as fh:
        json.dump(history, fh, indent=2, ensure_ascii=False)
    """

    return {
        "top_1_accuracy": top_1_accuracy,
        "top_10_accuracy": top_10_accuracy,
        "first_token_prob": first_token_prob,
        "second_token_prob": second_token_prob,
        "is_correct": is_correct_word,
    }


In [ ]:
#root_path   = "/content/drive/MyDrive/master_thesis/dataset_multilingual/linguistic/de"
#result_root = os.path.join(root_path, "result_new_t")

In [ ]:
#new_test_prompt("Das Gegenteil von ordentlich ist nachlässig.\nDas Gegenteil von anfänglich ist"," beendet", model, "adjective_antonym_1shot", result_root )

[Step 0] Gold: | be| Pred: | end|  Top‑1 Prob: 11.88%
[Step 1] Gold: |endet| Pred: |g|  Top‑1 Prob: 83.25%
[Step 2] Gold: |–| Pred: |ült|  Top‑1 Prob: 99.90%
[Step 3] Gold: |–| Pred: |ig|  Top‑1 Prob: 99.90%
[Step 4] Gold: |–| Pred: |.
|  Top‑1 Prob: 91.26%
Stop‑Token '.' erreicht – Generation beendet.


{'top_1_accuracy': 0.0,
 'top_10_accuracy': 0.0,
 'first_token_prob': 0.11883544921875,
 'second_token_prob': 0.10162353515625,
 'is_correct': False}

In [ ]:

def load_json_files(directory: str, selected_category: str, selected_relation: str) -> Dict[str, List[Dict]]:
    """
    Load JSON files from the specified directory.

    Args:
        directory (str): Directory containing the JSON files.

    Returns:
        dict: Dictionary where keys are relation names and values are lists of data entries.
    """
    data = {}
    for root, _, files in os.walk(directory):
        for file in files:
            #print(file)
            if file.endswith(".json"):
                path = os.path.join(root, file)
                #print(path)
                with open(path, 'r') as f:
                    category = os.path.basename(root)
                    #print("categorie",category)
                    if category == selected_category or not selected_category:
                        relation = os.path.basename(file).replace('.json', '')

                        if relation == selected_relation or not selected_relation:
                            if relation not in data:
                                data[relation] = []
                            data[relation].append(json.load(f))
    return data


def parse_samples(data: Dict[str, List[Dict]]) -> Tuple[List[str], List[str]]:
    facts = []
    targets = []
    sentences = []
    subjects = []
    for entries in data:
        #for entry in entries:
        prompt_templates = entries['prompt_templates']
        samples = entries['samples']

        for sample in samples:
            subject = sample['subject']
            obj = " " + sample['object']


            #for template in prompt_templates:  # Iterate through multiple templates
                #fact = template.format(subject)
            subjects.append(subject)
            targets.append(obj)
            #sentences.append(fact + obj)
            # fact = prompt_templates.format(subject)
            # facts.append(fact)
            # targets.append(obj)
    return prompt_templates, subjects, targets


In [ ]:
import os
import time

def calculate_average_accuracy(data, model, relation_name, permutation_name, base_output_dir):

    """Berechnet die durchschnittliche Top-1- und Top-10-Accuracy.
    Speichert automatisch Logits in einem passenden Ordner.

    Args:
        data (dict): Die geladenen Beispieldaten.
        model: Das Sprachmodell.
        relation_name (str): z. B. "adj_superlative"
        permutation_name (str): z. B. "permutation_2"
        base_output_dir (str): Basisverzeichnis für alle Resultate (z. B. "…/result/logits/")

    Returns:
        dict mit average_top_1_accuracy und average_top_10_accuracy.
    """
    start_time = time.time()
    total_top_1_accuracy = 0
    total_top_10_accuracy = 0
    total_examples = 0

    # Zielverzeichnis für Logits
    #json_output_folder = os.path.join(base_output_dir, permutation_name, relation_name)


    os.makedirs(base_output_dir, exist_ok=True)

    relation_base = relation_name.rsplit("_", 1)[0]
    json_output_folder = os.path.join(
         base_output_dir,
         relation_base
     )
    os.makedirs(json_output_folder, exist_ok=True)

    for key, value in data.items():
        prompt_templates, subjects, targets = parse_samples(value)

        for subj, target in zip(subjects, targets):
            for template in prompt_templates:
                prompt = template.format(subj)

                result = new_test_prompt(
                    prompt=prompt,
                    answer=target,
                    model=model,
                    relation_name=relation_name,
                    json_output_folder=json_output_folder,  # automatisch generiert!
                    prepend_space_to_answer=True,
                    prepend_bos=False,
                    print_details=False
                )

                total_top_1_accuracy += result["top_1_accuracy"]
                total_top_10_accuracy += result["top_10_accuracy"]
                total_examples += 1

    average_top_1_accuracy = total_top_1_accuracy / total_examples
    average_top_10_accuracy = total_top_10_accuracy / total_examples
    end_time = time.time()

    print(f"Dauer: {end_time - start_time:.2f}s | Beispiele: {total_examples}")
    return {
        "average_top_1_accuracy": average_top_1_accuracy,
        "average_top_10_accuracy": average_top_10_accuracy,
    }


In [ ]:
def save_accuracy_to_json(average_accuracies: dict, relation_name: str, output_folder: str) -> None:
    os.makedirs(output_folder, exist_ok=True)

    filename = f"accuracy_{relation_name.replace(' ', '_').lower()}.json"
    path = os.path.join(output_folder, filename)

    accuracy_data = {
        "relation": relation_name,
        "average_top_1_accuracy": round(average_accuracies.get("average_top_1_accuracy", 0), 2),
        "average_top_10_accuracy": round(average_accuracies.get("average_top_10_accuracy", 0), 2)
    }

    with open(path, "w", encoding="utf-8") as f:
        json.dump(accuracy_data, f, indent=2, ensure_ascii=False)

    print(f"Accuracy gespeichert unter: {path}")

In [ ]:
import re

# Basisverzeichnisse
root_path   =""
root_path   ="/content/drive/MyDrive/master_thesis/dataset_multilingual/factual/es"
result_root = os.path.join(root_path, "results")
logits_root = os.path.join(result_root, "logits/permutation_0")
eval_root   = os.path.join(result_root, "eval_accuracy")

# >>> Nur diese Permutation verarbeiten
target_permutation = "permutation_0"

perm_path = os.path.join(root_path, target_permutation)
if not os.path.isdir(perm_path):
    raise FileNotFoundError(f"{perm_path} existiert nicht oder ist kein Ordner")

json_files = [f for f in os.listdir(perm_path) if f.endswith(".json")]


for json_file in json_files:
    print(json_file)
    match = re.match(r"(.+?)_(\d+)shot\.json", json_file)
    if not match:
        continue

    relation_base, shot = match.groups()

    relation_name = f"{relation_base}_{shot}shot"

    logits_output_path   = logits_root
    accuracy_output_path = os.path.join(
        eval_root,
        target_permutation,
        f"accuracy_{relation_base}"
    )
    os.makedirs(accuracy_output_path, exist_ok=True)

    # Daten laden
    data = load_json_files(root_path, target_permutation, relation_name)
    #print(data)

    average_accuracies = calculate_average_accuracy(
        data=data,
        model=model,
        relation_name=relation_name,
        permutation_name=target_permutation,
        base_output_dir=logits_output_path
    )


    # Abspeichern
    save_accuracy_to_json(average_accuracies, relation_name, accuracy_output_path)
    print(f"fertig mit accuraccy_{json_file}")

print("Alle Accuracy Ferig")


person_band_lead_singer_3shot.json
Dauer: 4.73s | Beispiele: 21
Accuracy gespeichert unter: /content/drive/MyDrive/master_thesis/dataset_multilingual/factual/es/result_10_accuracy/eval_accuracy/permutation_4/accuracy_person_band_lead_singer/accuracy_person_band_lead_singer_3shot.json
fertig mit accuraccy_person_band_lead_singer_3shot.json
person_plays_position_in_sport_3shot.json
Dauer: 306.75s | Beispiele: 952
Accuracy gespeichert unter: /content/drive/MyDrive/master_thesis/dataset_multilingual/factual/es/result_10_accuracy/eval_accuracy/permutation_4/accuracy_person_plays_position_in_sport/accuracy_person_plays_position_in_sport_3shot.json
fertig mit accuraccy_person_plays_position_in_sport_3shot.json
person_band_lead_singer_4shot.json
Dauer: 4.77s | Beispiele: 21
Accuracy gespeichert unter: /content/drive/MyDrive/master_thesis/dataset_multilingual/factual/es/result_10_accuracy/eval_accuracy/permutation_4/accuracy_person_band_lead_singer/accuracy_person_band_lead_singer_4shot.json
fe

In [ ]:
relation_name = "city_in_country"

file_path_factual = "/content/drive/MyDrive/master_thesis/data/data_en/factual_data/zero_shot_data"
data = load_json_files(file_path_factual, "original_data", relation_name)
for key, value in data.items():
    #sentences, facts, targets = parse_samples(value)
    prompt_templates, subjects, targets = parse_samples(value)
    print(subjects)
    print(targets)
    print(prompt_templates)


['New York City', 'Rio de Janeiro', 'Buenos Aires', 'Mexico City', 'São Paulo', 'Los Angeles', 'Saint Petersburg', 'San Francisco', 'Ho Chi Minh City', 'Kuala Lumpur', 'Abu Dhabi', 'Cape Town', 'New Delhi', 'Las Vegas', 'Hong Kong', 'Tel Aviv', 'Johannesburg', 'Santo Domingo', 'Port-au-Prince', 'Santiago de Chile', 'Panama City', 'Siem Reap', 'Casablanca', 'San Juan', 'Costa Rica', 'Addis Ababa', 'Punta Cana']
[' United States', ' Brazil', ' Argentina', ' Mexico', ' Brazil', ' United States', ' Russia', ' United States', ' Vietnam', ' Malaysia', ' United Arab Emirates', ' South Africa', ' India', ' United States', ' China', ' Israel', ' South Africa', ' Dominican Republic', ' Haiti', ' Chile', ' Panama', ' Cambodia', ' Morocco', ' Puerto Rico', ' San José', ' Ethiopia', ' Dominican Republic']
['{} is part of', '{} is in the country of']


In [ ]:
def evaluate_templates_overall(
    subjects: List[str],
    targets: List[str],
    prompt_templates: List[str],
    model,
    second_token_threshold: float = 0.10
) -> Dict[str, Any]:
    """
    Evaluate all prompt templates and determine the best one based on overall performance,
    while identifying reliable and unreliable samples.

    Args:
        facts (list): List of facts (subject strings).
        targets (list): List of corresponding targets (answers).
        prompt_templates (list): List of prompt templates to test.
        model: The language model to evaluate.
        second_token_threshold (float): Maximum allowed probability for the second token.

    Returns:
        dict: A dictionary with:
            - template_scores: Overall scores and aggregated metrics for each template.
            - best_template: The best template based on the overall score.
            - best_template_metrics: Metrics for the best template.
            - reliable_samples: List of reliable samples with their details.
            - unreliable_samples: List of unreliable samples with their details.
    """
    template_scores = {template: {"first_token_prob_sum": 0,
                                  "second_token_valid_count": 0,
                                  "total_count": 0} for template in prompt_templates}


    reliable_samples = []
    unreliable_samples = []

    for subject, target in zip(subjects, targets):

        sample_details = []

        for template in prompt_templates:
            # Format the fact using the template
            is_reliable = False
            prompt = template.format(subject)

            # Evaluate the prompt
            res = new_test_prompt_(
                prompt=prompt,
                answer=target,
                model=model,
                print_details=True  # Suppress detailed printing for bulk processing
            )

            # Update metrics for the template
            if res["is_correct"]:
                template_scores[template]["first_token_prob_sum"] += res["first_token_prob"]
                template_scores[template]["total_count"] += 1
                if res["second_token_prob"] < second_token_threshold:
                    template_scores[template]["second_token_valid_count"] += 1
                    is_reliable = True
            else:
                template_scores[template]["first_token_prob_sum"] += 0
                template_scores[template]["total_count"] += 1

            # Collect sample details for each template
            sample_details.append({
                "template": template,
                "first_token_prob": res["first_token_prob"],
                "second_token_prob": res["second_token_prob"],
                "is_reliable": res["second_token_prob"] < second_token_threshold,
            })

            # Add sample to reliable or unreliable list
            if is_reliable:
                reliable_samples.append({"subject": subject, "object": target.strip()})
            else:
                unreliable_samples.append({"subject": subject, "object": target.strip()})

    # Calculate average scores and reliability for each template
    overall_scores = {}
    for template, metrics in template_scores.items():
        avg_first_token_prob = metrics["first_token_prob_sum"] / metrics["total_count"]
        reliability = metrics["second_token_valid_count"] / metrics["total_count"]
        overall_scores[template] = {
            "average_first_token_prob": avg_first_token_prob,
            "reliability": reliability,
            "overall_score": avg_first_token_prob * reliability,  # Combined score
        }

    # Find the best template based on the overall score
    best_template = max(overall_scores.items(), key=lambda x: x[1]["overall_score"])

    return {
        "template_scores": overall_scores,
        "best_template": best_template[0],
        "best_template_metrics": best_template[1],
        "reliable_samples": reliable_samples,
        "unreliable_samples": unreliable_samples,
    }


In [ ]:
def evaluate_best_template_samples(
    subjects: List[str],
    targets: List[str],
    best_template: str,
    model,
    first_token_threshold = 0.75,
    second_token_threshold: float = 0.10
) -> Dict[str, List[Dict]]:
    """
    Evaluate reliable and unreliable samples using the best template.

    Args:
        facts (list): List of facts (subject strings).
        targets (list): List of corresponding targets (answers).
        best_template (str): The best template determined earlier.
        model: The language model to evaluate.
        second_token_threshold (float): Maximum allowed probability for the second token.

    Returns:
        dict: A dictionary with:
            - reliable_samples: List of reliable samples in the specified format.
            - unreliable_samples: List of unreliable samples in the specified format.
    """
    reliable_samples = []
    unreliable_samples = []

    for subject, target in zip(subjects, targets):
        # Format the fact using the best template
        prompt = best_template.format(subject)

        # Evaluate the prompt
        res = new_test_prompt(
            prompt=prompt,
            answer=target,
            model=model,
            print_details=False  # Suppress detailed printing for bulk processing
        )

        # Determine reliability based on second token probability
        if res['is_correct'] == True and res["first_token_prob"] > first_token_threshold and res["second_token_prob"] < second_token_threshold:
            reliable_samples.append({"subject": subject, "object": target.strip()})
        else:
            unreliable_samples.append({"subject": subject, "object": target.strip()})

    return {
        "reliable_samples": reliable_samples,
        "unreliable_samples": unreliable_samples,
    }

In [ ]:
result=evaluate_templates_overall(subjects, targets, prompt_templates, model, second_token_threshold=0.10)


print("Template Scores:")
for template, metrics in result["template_scores"].items():
    print(f"Template: {template}")
    print(f"  Average First Token Prob: {metrics['average_first_token_prob']:.2f}")
    print(f"  Reliability: {metrics['reliability']:.2%}")
    print(f"  Overall Score: {metrics['overall_score']:.2f}")
    print("-" * 50)

    print(f"Best Template: {result['best_template']}")
    print(f"Best Template Metrics: {result['best_template_metrics']}")




def save_template_evaluation_to_json(result: dict, relation_name: str, output_folder: str) -> None:
    """
    Speichert die Template-Evaluationsergebnisse in einer JSON-Datei.

    Args:
        result (dict): Das Ergebnis von evaluate_templates_overall().
        relation_name (str): Name der Relation (für den Dateinamen).
        output_folder (str): Zielordner zum Speichern der Datei.
    """
    os.makedirs(output_folder, exist_ok=True)
    output_path = os.path.join(output_folder, f"template_eval_{relation_name.replace(' ', '_').lower()}.json")

    data_to_save = {
        "relation": relation_name,
        "template_scores": result.get("template_scores", {}),
        "best_template": result.get("best_template"),
        "best_template_metrics": result.get("best_template_metrics")
    }

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(data_to_save, f, indent=2, ensure_ascii=False)

    print(f"Evaluation gespeichert unter: {output_path}")



best_shots_scores="/content/drive/MyDrive/master_thesis/data/data_en/factual_data/zero_shot_data/best_template/best_template_file"

save_template_evaluation_to_json(
    result=result,
    relation_name=relation_name,
    output_folder=best_shots_scores
)

In [ ]:
best_template = result["best_template"]
print(f"Best Template: {best_template}")

# Evaluate reliable and unreliable samples for the best template
sample_evaluation_result = evaluate_best_template_samples(
    subjects=subjects,
    targets=targets,
    best_template=best_template,
    model=model,
    first_token_threshold = 0.75,
    second_token_threshold=0.10
)

In [ ]:
int(f"Reliable Examples: {sample_evaluation_result['reliable_samples']}")
print(f"Unreliable Examples: {sample_evaluation_result['unreliable_samples']}")